In [ ]:
import time
import pandas as pd
import json

import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

from __future__ import unicode_literals
import urllib

In [ ]:
lit = pd.read_csv(r'C:\Users\manokhina\Documents\thesis\top_cit_papers.csv', sep=';')

executable_path = r"C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe"
os.environ["webdriver.chrome.driver"] = executable_path

chrome_options = Options()
chrome_options.add_extension(r'C:\Program Files (x86)\Google\Chrome\Application\extension_3_9_0.crx')

In [ ]:
capabilities = DesiredCapabilities.CHROME.copy()
capabilities['unicodeKeyboard'] = 'True'

In [ ]:
driver = webdriver.Chrome(executable_path=executable_path, chrome_options=chrome_options)
driver.get("chrome-extension://omghfjlpggmjjaagoclmmobgdodcjboh/popup.html#")
driver.find_element_by_class_name('b-switch-label').click()

In [ ]:
pdf_links = {}

for string in lits:
    page_count = 1
    i = unicode(string.decode("utf-8")).split('&')[0]
    driver.get("http://scholar.google.com")
    
    time.sleep(2)
    search = driver.find_element_by_name("q") 
    search.send_keys(i)
    search.send_keys(Keys.RETURN)
    time.sleep(3)
    cit_list = driver.find_elements_by_partial_link_text(u"Цитируется")
    
    if cit_list:
        cit_list[0].click()
        
    urls = [u.get_attribute('href') for u in driver.find_elements_by_partial_link_text(u"[PDF]")]
    pdf_links[i] = urls
    page_count += 1
    driver.find_elements_by_link_text(unicode(str(page_count).decode("utf-8")))[0].click()

    while len(pdf_links[i]) < 50:  
        urls = [u.get_attribute('href') for u in driver.find_elements_by_partial_link_text(u"[PDF]")]
        pdf_links[i] += urls
        page_count += 1
        time.sleep(2)
        if driver.find_elements_by_link_text(unicode(str(page_count).decode("utf-8"))):
            driver.find_elements_by_link_text(unicode(str(page_count).decode("utf-8")))[0].click()
        else:
            break

In [ ]:
testfile = urllib.URLopener()

for k in pdf_links.keys():
    print k
    directory = os.path.join(r'C:\\Users\\manokhina\\Documents\\thesis\\data', k[:20])
    
    if not os.path.exists(directory):        
        os.makedirs(directory)
               
    for link in pdf_links[k]:       
        filename = link.split('/')[-1].split('.pdf')[0].replace("%",'')[:40] + '.pdf'        
        try:
            testfile.retrieve(link, os.path.join(directory, filename))
        except IOError  as e:
            continue       